In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
np.random.seed(54321)

!pip install ipdb
import ipdb

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


  Created wheel for ipdb: filename=ipdb-0.13.2-cp36-none-any.whl size=10522 sha256=84ff15ae55241f988a173ed7cbbd5b037a637274ddf4ffd7f7c4b554d9a29ec9
  Stored in directory: /root/.cache/pip/wheels/60/c2/15/793365e3c9318c46ba914263740d90f1fe67f544b979141ce4
Successfully built ipdb


In [2]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

# X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
# X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

In [3]:
# X_train=pd.read_csv('./data/train_data_preprocessing1.csv', sep=',') #we use this dataset to train our model
# Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
# X_test=pd.read_csv('./data/test_data_preprocessing1.csv', sep=',')

In [4]:
# X_test.head()

In [5]:
X_train.tail()

,Id,seq
1995,1995,TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...
1996,1996,ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...
1997,1997,TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...
1998,1998,CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...
1999,1999,TGCTAGTTGATGAAACAATAACTGCTAAAAGGTATACAGCCATGTC...


In [6]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)


In [7]:
# X_train['len'] = X_train.seq.apply(lambda x : len(x))

In [8]:
X_train.head()

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


In [9]:
import string
Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))

In [10]:
#  X_train['seq_1'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[0])
for i in range(0, 101, 3):
    X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
    X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])
    
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]]

#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+3])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+3])



In [11]:
X_train.head()

,Id,seq,seq_0,seq_3,seq_6,seq_9,seq_12,seq_15,seq_18,seq_21,seq_24,seq_27,seq_30,seq_33,seq_36,seq_39,seq_42,seq_45,seq_48,seq_51,seq_54,seq_57,seq_60,seq_63,seq_66,seq_69,seq_72,seq_75,seq_78,seq_81,seq_84,seq_87,seq_90,seq_93,seq_96,seq_99
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...,7,7,3,7,1,7,3,7,3,1,3,3,1,3,3,1,3,3,7,1,7,7,3,3,20,3,3,3,7,7,3,7,7,1
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...,3,3,7,7,3,1,20,7,3,1,20,7,7,1,1,7,1,7,20,1,3,1,3,3,3,7,1,1,7,7,20,3,7,7
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...,7,1,7,7,7,1,3,3,3,3,1,20,7,7,1,1,3,3,3,20,3,3,7,1,3,7,1,3,7,20,3,1,20,7
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...,7,20,3,7,1,1,7,7,3,20,20,1,7,7,7,7,7,7,3,7,3,3,20,7,1,3,1,20,3,7,1,7,3,1
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...,7,3,3,3,3,20,3,20,20,20,7,3,7,3,20,3,3,3,7,7,1,3,7,1,20,1,3,1,3,7,20,20,20,3


## What about counting number of different caracters 

In [12]:
# A, C, G, T

In [13]:
# X_train.seq.apply(lambda x : np.unique(x[:2]))


In [14]:
X = X_train.drop(['seq', 'Id'], axis=1)
X_t = X_test.drop(['seq', 'Id'], axis=1)
y = Y_train.Bound

## Models

In [15]:
# class LogisticRegressionBinary():
#     def __init__(self, lr=0.1, num_iter=100000, batch_size=1, verbose=False):
#         self.lr = lr
#         self.num_iter = num_iter
#         self.batch_size = batch_size
#         self.verbose = verbose
    
#     def __add_intercept(self, X):
#         intercept = np.ones((X.shape[0], 1))
#         return np.concatenate((intercept, X), axis=1)
    
#     def __sigmoid_func(self, z):
#         return 1 / (1 + np.exp(-z))
    
#     def __loss(self, h, y):
#         return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
#     def fit(self, X, y):
#         y = self.trans_y(y)
#         X = self.__add_intercept(X)
#         self.theta = np.zeros(X.shape[1])
        
#         for i in range(self.num_iter):
#             z = np.dot(X, self.theta)
#             h = self.__sigmoid_func(z)
                        
#             rand = np.random.choice(y.size, self.batch_size).squeeze()
#             gradient = np.dot(X[rand].T, (h[rand] - y[rand]))/y.size   
        
#             self.theta -= self.lr * gradient
#             #print('theta and grad',self.theta.shape ,  gradient.shape )
#             if(self.verbose == True and i % 100 == 0):
#                 z = np.dot(X, self.theta)
#                 h = self.__sigmoid(z)
#                 print(f'loss: {self.__loss(h, y)} \t')
    
#     def predict_probability(self, X):
#         X = self.__add_intercept(X)
    
#         return self.__sigmoid_func(np.dot(X, self.theta))
    
#     def predict(self, X, threshold=.5):
#           return np.where(self.predict_probability(X) >= 0.5, 1, 0)
        
          
#     def Accuracy_check(self,X,y):
#         return np.mean(self.predict(X)==y)
    
#     def trans_y(self, y):
#         if isinstance(y, pd.Series):
#             y = y.values
#         if isinstance(y, list):
#             y = np.array(y)
#         return y

In [16]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

# Kernel

In [22]:
import numpy as np

### Functions for you to fill in ###

def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    kernel_matrix = (X.dot(Y.T) + c)**p
    
    return kernel_matrix


def rbf_kernel_2(X, Y, gamma):
    """
        Compute the Gaussian RBF kernel between two matrices X and Y::
            K(x, y) = exp(-gamma ||x-y||^2)
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            gamma - the gamma parameter of gaussian function (scalar)

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    n, d = X.shape
    m = Y.shape[0]
    
    kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
    kernel_matrix = np.exp(-gamma*kernel_matrix)
    
    return kernel_matrix


def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

# def laplace(X1, X2, alpha=10):
#     return np.exp(-alpha*np.abs(X1-X2))

# def polynomial(X1, X2, d=2):
#     return (X1.dot(X2.T) +1)**d

def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def quadratic_kernel(X1, X2, power=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the quadratic kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return (1 + linear_kernel(X1, X2))**power

def rbf_poly_kernel(X1, X2, sigma=10, d=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    
    return K+(X1.dot(X2.T) +1)**d

In [23]:
class ksolveRR_2():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
        self.sample_weights = sample_weights
            
    
    def fit(self):
        if self.sample_weights not None:
            self.X *= self.sample_weights[:, None]
        
        X = self.X
        y = self.y
        lam = self.lam
        
        n, p = X.shape
        assert (len(y) == n)
        
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [24]:
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
            
    
    def fit(self):
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
            
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [25]:
import ipdb
# Logistic Ridge Regression (LRR)
class ksolveLRR():
    def __init__(self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        
        self.kernel = kernel
        
        self.sigma = sigma
        self.max_iter = max_iter
        self.tol = tol
        
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)

        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        K = self.kernel(X, X, self.sigma)

        # Initialize
        alpha = np.zeros(n)
        
        # Hint: Use IRLS
        for n_iter in range(self.max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*(f))
            
            alpha = ksolveRR_2(X, y, lam = 2*self.lam, \
                               sigma=self.sigma, sample_weights = w).fit().alpha
            
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < self.tol:
                break  
                
                
        self.n_iter = n_iter
        self.alpha = alpha
        
        return self
    
        
    def predict(self, X, threshold):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [27]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'

import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)
    return np.array(solution['x']).flatten()

solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]

def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

# SVM primal soft
class KernelSVM():
    def __init__(self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.C = C
        self.kernel = kernel
        self.lam = lam        
        self.sigma = sigma
        self.tol = tol
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)
        K = self.kernel(X, X, self.sigma)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        
       # Compute support vectors and bias b
        sv = np.logical_and((self.alpha>self.tol), (self.C - self.alpha > self.tol))
        self.bias = y[sv] - K[sv].dot(self.alpha*y)
        self.bias =  self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
        
        
    def predict(self, X, threshold):
#         y_pred = self.kernel(X, self.X_).dot(self.alphas* seld.y_)
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where((K_x.dot(self.alpha * self.y) +  self.bias) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [28]:
# # Logistic Ridge Regression (LRR)
# class perceptron():
#     def __init__(self, X, y, iteration=100):
#         self.theta = np.zeros(len(X))
#         self.X = X
#         self.y = y
#         self.iteration = iteration
#         self.theta_0 = 0
    
#     def fit(self):
        
#         X = self.X
#         y = self.y
        
#         n, p = X.shape
#         assert (len(y) == n)
        
#         for it in range(self.iteration):
#             for i in range(len(y)):
#                 if y[i]*(np.dot(self.theta, X[i]) + self.theta_0)<= 0 :
#                     self.theta = self.theta + (np.dot(y[i], X[i]))
#                     self.theta_0 += y[i]
    
        
#     def predict(self, X, threshold):
#         return np.where(X.dot(self.self.theta)+self.theta_0 >= threshold, 1, 0)
        
          
#     def Accuracy_check(self,X, y, threshold=.5):
#         return np.mean(self.predict(X, threshold)==y)

# Cross Validation

In [29]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TransformerM
# from sklearn.feature_extraction.text import TfidfTransformer

In [30]:
# vectorizer = CountVectorizer(max_features=2000)
# vectorizer.fit_transform(X[:10])
# # vectorizer.get_feature_names()

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

In [31]:
kfold=KFold(n_splits=5)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# X_cross = X.values
# X_t = X_t.values

y_cross = y.values

# vectorizer = TfidfVectorizer()
# X_cross = vectorizer.fit_transform(X)

X_cross = onehot_encoder.fit_transform(X.values[:, :101])
X_t_enc = onehot_encoder.fit_transform(X_t.values[:, :101])

# X_cross = onehot_encoder.fit_transform(X)
# X_t_enc = onehot_encoder.fit_transform(X_t)


# X_cross = transformer.fit_transform(X.values)
# X_t_enc = transformer.fit_transform(X_t.values)


# scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
# scaler.fit(X_cross)

# X_cross = scaler.transform(X_cross)

In [32]:
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.feature_selection import SelectKBest, chi2

In [33]:
# def feature_selector(X):
#     #sel = VarianceThreshold()
#     sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#     sel.fit_transform(X)
#     return X
# def best_feature_selector(X,y,num_features=50):
#     #print(X.shape)
#     features = SelectKBest(chi2, k=num_features).fit(X, y)
#     #print(X_new.shape)
#     return features

In [34]:
X.shape

(2000, 34)

In [35]:
# trans = best_feature_selector(X, y, 30)

In [36]:
# X = trans.transform(X)
# X_t = trans.transform(X_t)

In [44]:
X.values.shape

(2000, 34)

In [45]:
X_t.values.shape

(1000, 34)

In [46]:
X_t_enc.shape

(1000, 136)

In [47]:
X_cross.shape

(2000, 136)

In [48]:
from sklearn import linear_model as lm
from sklearn.metrics import accuracy_score, roc_auc_score

# from np.line

In [49]:
# # Simulate data
# np.random.seed(54321)
# p = 2

# def generate_Xy(n_samples, p=101, sigma=.2):
#     # Half of y is 1s, other half is -1
#     y = np.ones(n_samples)
#     mid = int(n_samples / 2)
#     y[mid:] *= -1
    
#     X = np.random.normal(0, sigma, (n_samples, p))
#     X += (1 - y[:, np.newaxis]) / 2 # add 1 when y = -1
#     # X of shape (n, p)
#     # y vector of length n
#     return X, y

# # Training data
# X_train, y_train = generate_Xy(20)

# # Testing data
# X_cross, y_cross = generate_Xy(2000)

# y_cross = np.where(y_cross==1, 1, 0)

In [50]:
from collections import defaultdict
results = defaultdict(float)

In [52]:
!pip install optuna
import optuna

     |████████████████████████████████| 184kB 2.8MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 7.6MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.7MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 7.3MB/s  eta 0:00:01
     |████████████████████████████████| 112kB 6.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 6.8MB/s  eta 0:00:01
     |████████████████████████████████| 51kB 6.4MB/s  eta 0:00:01
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=8778657d8f5515ca8674c4ff62f94f8a3b8f364553cfa5eda1d3df13bb21c1c3
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.4.0-cp36-none-any.whl size=254554 sha256=6e31860d299c6183d46a5f47ed466aa57f8a0

In [ ]:
# def objective_sgd(trial):
#     sigma  = trial.suggest_int('sigma', 1, 20)
# #     d  = trial.suggest_int('c', 1, 20)
#     lam = trial.suggest_loguniform('lam', 1e-10, 1e-0)
    
#     models = {ksolveRR_2 : 'Kernal Ridge Regression'}
    
#     for model in models:
#         accuracy = []
#         for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
#             X_train, y_train = X_cross[train_index], y_cross[train_index]
#             X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
#             model_curr = model(X_train, y_train, lam, sigma)
#             model_curr.fit()
#             accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
#             print(f'accurracy fold {i}: {accuracy[i]}')
        
#         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

#     return np.mean(accuracy)

# sampler = optuna.samplers.TPESampler()
# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective_sgd, n_trials=500,show_progress_bar=True)

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# def objective_sgd(trial):
# #     c  = trial.suggest_int('c', 1, 20)
#     sigma  = trial.suggest_loguniform('sigma', 1, 20)
#     lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
    
#     models = {ksolveRR : 'Kernal Ridge Regression'}
    
#     for model in models:
#         accuracy = []
#         for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
#             X_train, y_train = X_cross[train_index], y_cross[train_index]
#             X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
#             model_curr = model(X_train, y_train, lam, sigma)
#             model_curr.fit()
#             accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
#             print(f'accurracy fold {i}: {accuracy[i]}')
        
#         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

#     return np.mean(accuracy)

# sampler = optuna.samplers.TPESampler()
# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective_sgd, n_trials=500,show_progress_bar=True)


# trial = study.best_trial

# print('Accuracy: {}'.format(trial.value))
# print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [53]:
def objective_sgd(trial):
    c  = trial.suggest_int('c', 1, 20)
    sigma  = trial.suggest_int('sigma', 1, 30)
    kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
    lam = trial.suggest_loguniform('lam', 1e-18, 1e-0)
    tol = trial.suggest_loguniform('tol', 1e-5, 1e-0)
    
    # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
    # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
    # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
    # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
    models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
              ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM', } 
              
    for model in models:
        accuracy = []
        for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
            X_train, y_train = X_cross[train_index], y_cross[train_index]
            X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
            
            
            
            if model == ksolveRR_2:
                sample_weights = np.random.rand(len(y_train))
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
                
            elif model == ksolveLRR:
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
            else:
                model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
            model_curr.fit()
            accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
            print(f'accurracy fold {i}: {accuracy[i]}')
        
        print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

    return np.mean(accuracy)

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective_sgd, n_trials=100,show_progress_bar=True)


trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7f3f0f2528c8> which is of type function.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7f3f0f252048> which is of type function.



[W 2020-05-27 22:19:18,791] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7f3f0f2528c8> which is of type function.
[W 2020-05-27 22:19:18,793] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7f3f0f252048> which is of type function.
[W 2020-05-27 22:19:18,798] Setting status of trial#0 as TrialState.FAIL because of the following error: TypeError("__init__() got an unexpected keyword argument 'C'",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 699, in _run_trial
    result = func(trial)
  File "<ipython-input-53-460d724618e2>", line 30, in objective_sgd
    model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
TypeError: __init__() got an unexpected keyword argument 'C'



TypeError: __init__() got an unexpected keyword argument 'C'

In [ ]:
def objective_sgd(trial):
    c  = trial.suggest_loguniform('c', 1, 20)
    sigma  = trial.suggest_loguniform('sigma', 1, 20)
    lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
    tol = trial.suggest_loguniform('tol', 1e-5, 1e-0)
    
    models = {KernelSVM : 'Kernal SVM'}
    
    for model in models:
        accuracy = []
        for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
            X_train, y_train = X_cross[train_index], y_cross[train_index]
            X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
            
            model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol = tol)
            
            model_curr.fit()
            accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
            print(f'accurracy fold {i}: {accuracy[i]}')
        
        print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

    return np.mean(accuracy)

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective_sgd, n_trials=500,show_progress_bar=True)


trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



     pcost       dcost       gap    pres   dres
 0: -1.4269e+03 -7.6772e+03  1e+04  6e-01  2e-15
 1: -1.5497e+03 -2.5751e+03  1e+03  6e-03  8e-16
 2: -2.0419e+03 -2.0616e+03  2e+01  7e-05  4e-16
 3: -2.0468e+03 -2.0470e+03  2e-01  7e-07  6e-16
 4: -2.0468e+03 -2.0468e+03  2e-03  7e-09  6e-16
Optimal solution found.
accurracy fold 0: 0.4875


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: RuntimeWarning:

invalid value encountered in greater_equal



     pcost       dcost       gap    pres   dres
 0: -1.4375e+03 -7.7041e+03  1e+04  6e-01  2e-15
 1: -1.5612e+03 -2.5917e+03  1e+03  6e-03  5e-16
 2: -2.0570e+03 -2.0768e+03  2e+01  7e-05  6e-16
 3: -2.0620e+03 -2.0622e+03  2e-01  7e-07  5e-16
 4: -2.0620e+03 -2.0620e+03  2e-03  7e-09  4e-16
Optimal solution found.
accurracy fold 1: 0.4725
     pcost       dcost       gap    pres   dres
 0: -1.3845e+03 -7.5696e+03  1e+04  6e-01  2e-15
 1: -1.5036e+03 -2.5089e+03  1e+03  6e-03  7e-16
 2: -1.9811e+03 -2.0005e+03  2e+01  7e-05  5e-16
 3: -1.9859e+03 -1.9861e+03  2e-01  7e-07  7e-16
 4: -1.9859e+03 -1.9859e+03  2e-03  7e-09  5e-16
 5: -1.9859e+03 -1.9859e+03  2e-05  7e-11  5e-16
Optimal solution found.
accurracy fold 2: 0.5475
     pcost       dcost       gap    pres   dres
 0: -1.4216e+03 -7.6637e+03  1e+04  6e-01  2e-15
 1: -1.5439e+03 -2.5669e+03  1e+03  6e-03  7e-16
 2: -2.0343e+03 -2.0539e+03  2e+01  7e-05  5e-16
 3: -2.0391e+03 -2.0393e+03  2e-01  7e-07  4e-16
 4: -2.0392e+03 -2.0392

In [40]:
# models = {solveRR: 'Ridge Regression (RR)', solveWRR:'Weighted Ridge Regression (WRR)', \
#           solveLRR : 'Logistic Ridge Regression (LRR)', ksolveRR : 'Kernal Ridge Regression'}

models = {KernelSVM : 'Kernal Ridge Regression'}
for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if model == solveWRR:
            w = np.random.rand(len(y_train))
            model_curr = solveWRR(X_train, y_train, w, lam=0.01)
        else:
#             model_curr = model(X_train, y_train, lam=0.09)
#             model_curr = model(X_train, y_train, 7.804693708068893e-05, 4.054424533679082)
    
            model_curr = model(X_train, y_train, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

     pcost       dcost       gap    pres   dres
 0: -4.4385e+02 -2.7639e+02  7e+03  2e+01  4e-16
 1: -6.8600e+01 -2.5590e+02  3e+02  2e-01  4e-16
 2: -7.4384e+01 -8.7337e+01  1e+01  9e-03  7e-16
 3: -8.0637e+01 -8.0776e+01  2e-01  9e-05  2e-16
 4: -8.0699e+01 -8.0701e+01  2e-03  9e-07  2e-16
 5: -8.0700e+01 -8.0700e+01  2e-05  9e-09  4e-16
Optimal solution found.
accurracy fold 0: 0.4875


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in greater_equal


     pcost       dcost       gap    pres   dres
 0: -4.4715e+02 -2.7672e+02  7e+03  2e+01  4e-16
 1: -6.9110e+01 -2.5629e+02  3e+02  2e-01  5e-16
 2: -7.4937e+01 -8.7898e+01  1e+01  9e-03  7e-16
 3: -8.1237e+01 -8.1376e+01  2e-01  9e-05  4e-16
 4: -8.1299e+01 -8.1301e+01  2e-03  9e-07  2e-16
 5: -8.1300e+01 -8.1300e+01  2e-05  9e-09  2e-16
Optimal solution found.
accurracy fold 1: 0.4725
     pcost       dcost       gap    pres   dres
 0: -4.3065e+02 -2.7507e+02  7e+03  2e+01  4e-16
 1: -6.6560e+01 -2.5434e+02  3e+02  2e-01  4e-16
 2: -7.2173e+01 -8.5096e+01  1e+01  9e-03  6e-16
 3: -7.8239e+01 -7.8378e+01  2e-01  9e-05  2e-16
 4: -7.8299e+01 -7.8301e+01  2e-03  9e-07  2e-16
 5: -7.8300e+01 -7.8300e+01  2e-05  9e-09  3e-16
Optimal solution found.
accurracy fold 2: 0.5475
     pcost       dcost       gap    pres   dres
 0: -4.4220e+02 -2.7622e+02  7e+03  2e+01  4e-16
 1: -6.8345e+01 -2.5570e+02  3e+02  2e-01  4e-16
 2: -7.4108e+01 -8.7057e+01  1e+01  9e-03  6e-16
 3: -8.0337e+01 -8.0476

In [ ]:
# models = {solveRR: 'Ridge Regression (RR)', solveWRR:'Weighted Ridge Regression (WRR)', \
#           solveLRR : 'Logistic Ridge Regression (LRR)', ksolveRR : 'Kernal Ridge Regression'}

models = {ksolveLRR : 'Kernal Ridge Regression'}
for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if model == solveWRR:
            w = np.random.rand(len(y_train))
            model_curr = solveWRR(X_train, y_train, w, lam=0.01)
        else:
#             model_curr = model(X_train, y_train, lam=0.09)
            model_curr = model(X_train, y_train, lam = 20, sigma = 9, max_iter=50, tol=1e-9)
#             model_curr = model(X_train, y_train)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

In [ ]:
# 0.6535 = 0.68800
# 0.657 = 0.69200

In [ ]:
# Cehckinf full model
model = ksolveRR(X_cross, y_cross, 0.00000000001, 4)
# model = svm_primal_soft_to_qp(X_cross, y_cross, C=1)

model.fit()

model.Accuracy_check(X_cross, y_cross, threshold=0.5)

# Predictions

In [ ]:
model = ksolveRR(X_cross, y_cross, 7.804693708068893e-05, 4.054424533679082)
model.fit()
y_pred = model.predict(X_t_enc, 0.5)

In [ ]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

In [ ]:
sample['Bound'] = y_pred

In [ ]:
sample.tail()

In [ ]:
sample.to_csv('./ksolveRR_68_cv_7.804693708068893e-05_4.054424533679082.csv', index=False)